# 🗂️ Week 07-08 · Notebook 14 · Agent State Management & Persistence

Persist agent memory and checkpoints to support multi-shift handoffs and compliance reviews.

## 🎯 Learning Objectives
- Implement memory modules (conversation buffer, vectorstore-backed memory).
- Persist agent state to Redis / PostgreSQL for cross-shift continuity.
- Manage checkpointing and rollback strategies.
- Enforce retention policies to comply with IT and legal standards.

## 🧩 Scenario
Night shift escalates a complex issue. Morning shift must resume with full context while ensuring sensitive data is retained only for 30 days.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.schema import HumanMessage, AIMessage

# --- 1. Basic Conversation Buffer Memory ---
# This memory keeps a simple list of the conversation turns.

# Initialize memory to store the conversation
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Initialize the LLM and the conversation chain
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)
chain = ConversationChain(llm=llm, memory=memory, verbose=True)

# --- Simulate a conversation during a shift ---
print("--- Shift 1 Conversation ---")
chain.predict(input='Technician on duty. We just replaced the spindle bearings on Press-14, but vibration is still high.')
chain.predict(input='What was the root cause the last time this happened, around 6 months ago?')

# --- 2. Inspect the Memory ---
# At the end of the shift, we can see what's stored in memory.
print("\n--- Memory State at End of Shift 1 ---")
memory_state = memory.load_memory_variables({})
for message in memory_state['chat_history']:
    print(f"[{message.__class__.__name__}]: {message.content}")

# --- 3. Simulating a Shift Handoff ---
# To hand off, we could serialize the memory content. For this demo, we'll just create a new chain
# and manually load the previous conversation history.

# New shift, new chain, but with the *same memory object*
print("\n--- Shift 2 Begins ---")
chain_shift2 = ConversationChain(llm=llm, memory=memory, verbose=True)

# The new technician can now ask questions with full context.
chain_shift2.predict(input="This is the morning shift taking over. Based on the previous conversation, what's the next logical troubleshooting step?")

print("\n--- Final Memory State ---")
final_memory_state = memory.load_memory_variables({})
for message in final_memory_state['chat_history']:
    print(f"[{message.__class__.__name__}]: {message.content}")

### 🧠 Vectorstore-Backed Memory
Use embeddings to recall similar incidents across shifts.

In [ ]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from datetime import datetime, timedelta

# --- VectorStore-Backed Memory for Long-Term Recall ---
# This type of memory stores conversation snippets in a vector store, allowing the agent
# to recall semantically similar past events, even from different shifts or years ago.

# 1. Setup the Vector Store and Embedding function
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# Using a persistent directory for the vector store
vectorstore = Chroma(
    collection_name='long_term_agent_memory', 
    embedding_function=embedding,
    persist_directory='./agent_memory_db'
)

# 2. Create the RetrieverMemory
# This connects the vector store to the memory interface.
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
retriever_memory = VectorStoreRetrieverMemory(retriever=retriever)

# 3. Save context from past incidents (simulating historical data)
# This would happen continuously as the agent operates.
retriever_memory.save_context(
    {'input': 'Press 14 had a bearing failure due to improper lubrication.'}, 
    {'output': 'Resolved by changing lubricant and updating SOP-122.'}
)
retriever_memory.save_context(
    {'input': 'CNC-08 showed high vibration after a software update.'},
    {'output': 'Resolved by rolling back the update and recalibrating the tool head.'}
)
retriever_memory.save_context(
    {'input': 'Press 14 had another vibration issue, this time caused by a loose mounting bolt.'},
    {'output': 'Resolved by tightening bolts to spec.'}
)

# 4. Use the memory to recall relevant context for a new problem
new_incident_query = 'Press 14 is vibrating again after we did some maintenance.'
relevant_history = retriever_memory.load_memory_variables({'input': new_incident_query})

print(f"--- New Incident ---")
print(f"Query: '{new_incident_query}'")
print("\n--- Relevant Historical Context Recalled from Vector Memory ---")
# The 'history' key contains the most relevant past conversation snippet.
print(relevant_history.get('history'))

## 🧾 Retention Policy
- Conversation buffers retained 30 days (auto-purge).
- Vectorstore entries tagged with `retire_after` timestamp.
- Manual purge process documented for legal holds.

## 🧪 Lab Assignment
1. Add Redis backend for memory persistence and simulate restart recovery.
2. Implement checkpoint snapshots saved to Cloud Storage with checksum.
3. Design retention job that purges expired entries daily and logs results.
4. Present memory architecture to IT for approval.

## ✅ Checklist
- [ ] Memory modules implemented
- [ ] Persistence layer configured
- [ ] Retention policy documented
- [ ] Lab deliverables reviewed

## 📚 References
- LangChain Memory Guide
- Corporate Data Retention Policy
- Redis Persistence Best Practices